In [106]:
from openai import OpenAI

In [107]:
from dotenv import load_dotenv

In [108]:
from supabase import create_client, Client

In [109]:
import anthropic

In [110]:
load_dotenv()

True

In [111]:
import os

In [112]:
import requests

In [114]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [115]:
anthropic_api_key = os.getenv("CLAUDE_API_KEY")

In [136]:
client_anthropic = anthropic.Anthropic(api_key = anthropic_api_key)

In [264]:
import json

In [22]:
supabase_url = os.environ.get("SUPABASE_URL")

In [23]:
supabase_api_key = os.environ.get("SUPBASE_KEY")

In [25]:
supabase: Client = create_client(supabase_url, supabase_api_key)

In [27]:
sql_query

'\nSELECT * FROM webhooks order by created_at ASC\n'

In [283]:
result = supabase.table('webhooks')\
    .select('id, news_output,source_urls, news_date')\
    .order('created_at', desc=False)\
    .execute()

result.data

[{'id': 2,
  'news_output': 'Reuters (Jan 24, 2026) reports the Singapore government will invest over S$1 billion (about $778.8 million) in public AI research through 2030. The funding targets priority research areas including responsible and resource-efficient AI, talent development from pre-university to faculty levels, and building capabilities to support industry adoption of AI. The announcement builds on prior Singapore commitments (e.g., 2024 HPC funding and AI Singapore investments) and is intended to strengthen national AI capabilities and global competitiveness.',
  'source_urls': ['https://www.reuters.com/world/asia-pacific/singapore-invest-over-779-million-public-ai-research-through-2030-2026-01-24/'],
  'news_date': '2026-01-24'},
 {'id': 3,
  'news_output': "TechCrunch (Jan 24, 2026) published an analysis titled 'A new test for AI labs: Are you even trying to make money?' The piece proposes a five-level scale to rate AI labs by commercial ambition and summarizes recent fun

In [292]:
def prompt_generator_research():
    formatted_prompt = []
    
    for item in result.data:
        event_id = f"{item['id']}_{item['news_date']}"
        prompt = f"""You are given news summary: {item['news_output']} and source: {item['source_urls'][0]}, you are supposed to run a deep research and read other sources as well. And then give a well structured ouput in the form of points and sources."""

        prompt_dict = {
            'event_id': event_id,
            'prompt': prompt
        }
        
        formatted_prompt.append(prompt_dict)

    return formatted_prompt

In [293]:
prompt_generator_research()

[{'event_id': '2_2026-01-24',
  'prompt': 'You are given news summary: Reuters (Jan 24, 2026) reports the Singapore government will invest over S$1 billion (about $778.8 million) in public AI research through 2030. The funding targets priority research areas including responsible and resource-efficient AI, talent development from pre-university to faculty levels, and building capabilities to support industry adoption of AI. The announcement builds on prior Singapore commitments (e.g., 2024 HPC funding and AI Singapore investments) and is intended to strengthen national AI capabilities and global competitiveness. and source: https://www.reuters.com/world/asia-pacific/singapore-invest-over-779-million-public-ai-research-through-2030-2026-01-24/, you are supposed to run a deep research and read other sources as well. And then give a well structured ouput in the form of points and sources.'},
 {'event_id': '3_2026-01-24',
  'prompt': "You are given news summary: TechCrunch (Jan 24, 2026) p

In [182]:
system_prompt = prompt_generator_research()

In [180]:
def openai_research():
    research_output = []
    for i in system_prompt:
        print(i)
        response = client.responses.create(
            model = "gpt-4o",
            # reasoning = {"effort": "high"},
            tools = [{
                "type": "web_search"
            }],

            include = ["web_search_call.action.sources"],
            input = i,
            # max_output_tokens = 1024
        )

        output = response.output_text

        research_output.append(output)

    return research_output

In [181]:
openai_research()

You are given news summary: Reuters (Jan 24, 2026) reports the Singapore government will invest over S$1 billion (about $778.8 million) in public AI research through 2030. The funding targets priority research areas including responsible and resource-efficient AI, talent development from pre-university to faculty levels, and building capabilities to support industry adoption of AI. The announcement builds on prior Singapore commitments (e.g., 2024 HPC funding and AI Singapore investments) and is intended to strengthen national AI capabilities and global competitiveness. and source: https://www.reuters.com/world/asia-pacific/singapore-invest-over-779-million-public-ai-research-through-2030-2026-01-24/, you are supposed to run a deep research and read other sources as well. And then give a well structured ouput in the form of points and sources.


["Here is a **well-structured, in-depth analysis** (approximately 900 words), summarizing and adding context through multiple reputable sources, with each statement duly cited. I present the details in clear bullet-point sections for readability, as requested:\n\n---\n\n### • Overview of the Announcement\n\n- On **January 24, 2026**, Reuters reported that the **Singapore government will commit over S$1\u202fbillion (approximately US$778.8\u202fmillion)** to support **public AI research through 2030**, aiming to enhance national AI capabilities and global competitiveness ([nst.com.my](https://www.nst.com.my/amp/world/region/2026/01/1363738/singapore-invest-over-us779-million-public-ai-research-through-2030?utm_source=openai)).\n- Government statements clarify this sum is allocated under the **National AI Research and Development Plan (NAIRD)** spanning **2025 to 2030**, and is part of broader efforts under the updated **National AI Strategy (NAIS 2.0)** ([channelnewsasia.com](https://ww

In [281]:
def openai_article_builder(researched_data):
    final_news_article = []
    for i in researched_data:
        response = client.responses.create(
            model = "gpt-4.1",
            input = f""" 
            You are a professional tech journalist for "The AI Times" newspaper.

            Your task: Write a comprehensive news article covering today's AI developments.
            
            Entire news story and its details: {i}
            
            INSTRUCTIONS:
            1. Write a newspaper-quality article (800-1000 words)
            2. Create an engaging headline
            3. Use journalistic style: objective, clear, professional and opinionated
            4. Include all the sources & links at the end and not ANY sources or links during the article (THIS IS NON NEGOTIABLE)

            You are ONLY supposed to refer to the sources and links and NOT supposed to hallucinate. 
            
            However, the opinion you have be your own but it should be based on the above article. 
            """
        )


        final_output = response.output_text

        json_format = json_formatter(final_output, "openai")

        final_news_article.append(json_format)

    return final_news_article

In [266]:
b = ["Here is a **well-structured, in-depth analysis** (approximately 900 words), summarizing and adding context through multiple reputable sources, with each statement duly cited. I present the details in clear bullet-point sections for readability, as requested:\n\n---\n\n### • Overview of the Announcement\n\n- On **January 24, 2026**, Reuters reported that the **Singapore government will commit over S$1\u202fbillion (approximately US$778.8\u202fmillion)** to support **public AI research through 2030**, aiming to enhance national AI capabilities and global competitiveness ([nst.com.my](https://www.nst.com.my/amp/world/region/2026/01/1363738/singapore-invest-over-us779-million-public-ai-research-through-2030?utm_source=openai)).\n- Government statements clarify this sum is allocated under the **National AI Research and Development Plan (NAIRD)** spanning **2025 to 2030**, and is part of broader efforts under the updated **National AI Strategy (NAIS 2.0)** ([channelnewsasia.com](https://www.channelnewsasia.com/singapore/singapore-invest-over-1-billion-in-national-ai-research-plan-5875861?utm_source=openai)).\n\n---\n\n### • Key Objectives and Strategic Focus Areas\n\nThe NAIRD investment is structured around **three main pillars**:\n\n1. **Fundamental AI Research**  \n   - Establishment of **Research Centres of Excellence (RCEs)** within public research institutions, targeting “long‑term, difficult questions” ([thedailystar.net](https://www.thedailystar.net/news/asia/southeast-asia/news/singapore-invest-785-million-public-research-ai-4089316?utm_source=openai)).  \n   - RCEs will focus on:\n     - **Resource-efficient AI**, given the high resource demands of training and inference and Singapore’s dense data centre infrastructure ([channelnewsasia.com](https://www.channelnewsasia.com/singapore/singapore-invest-over-1-billion-in-national-ai-research-plan-5875861?utm_source=openai)).  \n     - **Responsible AI**, emphasizing security, ethical safeguards, and protection against misuse ([channelnewsasia.com](https://www.channelnewsasia.com/singapore/singapore-invest-over-1-billion-in-national-ai-research-plan-5875861?utm_source=openai)).  \n     - **Emerging AI methodologies** and **general-purpose AI**, working toward versatile systems capable of multi-domain tasks (e.g., reading research papers, analyzing proteins) ([sg.news.yahoo.com](https://sg.news.yahoo.com/singapore-invest-1-billion-over-115709702.html/?utm_source=openai)).\n\n2. **Applied AI Research and Industry Adoption**  \n   - Building capacity for **industry-wide adoption** of AI across domains like manufacturing, trade, healthcare, urban solutions, and sustainability ([sg.news.yahoo.com](https://sg.news.yahoo.com/singapore-invest-1-billion-over-115709702.html/?utm_source=openai)).  \n   - Strengthening **AI engineering capabilities** for practical deployment: bridging research-to-system integration ([channelnewsasia.com](https://www.channelnewsasia.com/singapore/singapore-invest-over-1-billion-in-national-ai-research-plan-5875861?utm_source=openai)).  \n   - Collaborating with industry players—including **Changi Airport Group** and **Sembcorp**—to co-develop real-world applications ([businesstimes.com.sg](https://www.businesstimes.com.sg/companies-markets/singapore-doubles-down-ai-s1b-rd-push-amid-us-china-dominance?utm_source=openai)).  \n   - Using venues like **Jewel at Changi Airport** as showcases of AI applications such as security screening, automated baggage handling, and robotic inspections ([sg.news.yahoo.com](https://sg.news.yahoo.com/singapore-invest-1-billion-over-115709702.html/?utm_source=openai)).\n\n3. **Talent Development Across the Pipeline**  \n   - Expanding AI talent from pre-university (e.g., the National Olympiad in AI) through post-graduate and faculty levels ([thedailystar.net](https://www.thedailystar.net/news/asia/southeast-asia/news/singapore-invest-785-million-public-research-ai-4089316?utm_source=openai)).  \n   - Scaling programmes like the **AI Singapore PhD Fellowship**, the **AI Accelerated Masters Programme**, and the **AI Visiting Professorship**—which has already supported eight awardees—to cultivate research talent ([channelnewsasia.com](https://www.channelnewsasia.com/singapore/singapore-invest-over-1-billion-in-national-ai-research-plan-5875861?utm_source=openai)).  \n   - Enhancing scholarships, research opportunities, and exposure to international institutions for students to compete globally ([channelnewsasia.com](https://www.channelnewsasia.com/singapore/singapore-invest-over-1-billion-in-national-ai-research-plan-5875861?utm_source=openai)).\n\n---\n\n### • Continuity and Integration with Existing Investments\n\n- This S$1\u202fbillion investment builds upon prior AI-related investments:\n  - **S$500 million (2024)** for securing **high-performance compute infrastructure** to support AI innovation ([smartnation.gov.sg](https://www.smartnation.gov.sg/media-hub/press-releases/20240301a/?utm_source=openai)).\n  - Over **S$500 million** through **AI Singapore**, a national AI R&D programme targeting capability-building ([nst.com.my](https://www.nst.com.my/amp/world/region/2026/01/1363738/singapore-invest-over-us779-million-public-ai-research-through-2030?utm_source=openai)).\n- It is also tied into the larger **S$37\u202fbillion RIE 2030 plan** launched in December 2025, which earmarks funding across research, innovation, talent, and sector-specific transformation ([thestar.com.my](https://www.thestar.com.my/aseanplus/aseanplus-news/2025/12/05/singapore-commits-record-s37bil-to-drive-research-innovation-and-enterprise-over-next-five-years?utm_source=openai)).\n- The integration ensures alignment of AI-specific R&D with Singapore’s broader ecosystem of **Research, Innovation, and Enterprise (RIE)** efforts ([channelnewsasia.com](https://www.channelnewsasia.com/singapore/singapore-invest-over-1-billion-in-national-ai-research-plan-5875861?utm_source=openai)).\n\n---\n\n### • Strategic Intent: Positioning Singapore in the Global AI Landscape\n\n- Singapore’s R&D focus seeks to carve a niche—**aiming for depth and relevance rather than direct competition** with giants like the US or China ([businesstimes.com.sg](https://www.businesstimes.com.sg/companies-markets/singapore-doubles-down-ai-s1b-rd-push-amid-us-china-dominance?utm_source=openai)).\n- The emphasis on **resource efficiency**, **responsibility**, and **public-good AI** reflects Singapore’s unique constraints (energy, water, ecosystem density) and ambition to lead in sustainable AI innovation ([channelnewsasia.com](https://www.channelnewsasia.com/singapore/singapore-invest-over-1-billion-in-national-ai-research-plan-5875861?utm_source=openai)).\n- Prior performance: in **2025**, Singapore ranked **3rd globally** in The Observer’s **Global AI Index**, trailing only the US and China ([channelnewsasia.com](https://www.channelnewsasia.com/singapore/singapore-invest-over-1-billion-in-national-ai-research-plan-5875861?utm_source=openai)).\n\n---\n\n### • Summary Table (Bullet Style Recap)\n\n- **Announcement Date**: January 24, 2026.  \n- **Funding Amount**: Over S$1\u202fbillion (~US$779–786\u202fmillion).  \n- **Timeframe**: 2025–2030.  \n- **Strategic Pillars**:\n  1. Fundamental AI Research (RCEs, responsible and resource-efficient AI, emerging & general-purpose AI)  \n  2. Applied AI & Industry Adoption (engineering capabilities, sector deployment, real-world labs)  \n  3. Talent Development (pre-university to faculty, fellowships, visiting professorships)  \n- **Builds On**:\n  - 2024 investments in AI compute and AI Singapore.  \n  - S$37\u202fbillion RIE 2030 overarching research strategy.  \n- **Strategic Goal**: Enhance Singapore’s role as a global AI innovation hub with sustainable, responsible, and impactful AI research.\n\n---\n\n### • Final Thoughts\n\nSingapore’s S$1\u202fbillion AI R&D pledge (announced January 24, 2026) represents a **significant, multi-sectoral effort** to deepen AI expertise, integrate AI into industry, and develop talent across all levels. It underscores Singapore’s commitment to **sustainable and responsible innovation**, and to building a **globally competitive but strategically differentiated AI ecosystem**.\n\nIf you'd like, I can further explore specific programmes (e.g., AI Visiting Professorship, AI Accelerated Masters), compare Singapore’s approach with other nations, or analyze potential outcomes. Just let me know!\n\n---\n\nLet me know if you’d like a particular section expanded with additional depth or links to other policies or programmes."]


openai_article_builder(b)

[{'model': 'openai',
  'headline': 'Singapore Bets Big on AI: S$1 Billion Investment Aims to Secure Sustainable Leadership in Artificial Intelligence',
  'content': 'January 24, 2026 — In a landmark announcement set to reverberate across the global technology landscape, the Singaporean government has pledged over S$1 billion (about US$779 million) to public artificial intelligence (AI) research through 2030. Through this ambitious National AI Research and Development Plan (NAIRD), authorities are sending a strong signal: Singapore intends not just to participate in, but to shape, the future of AI.\n\nA Strategic Commitment: S$1 Billion to Catalyze National AI Ambitions\n\nSingapore’s move, communicated through government statements this morning, cements AI at the heart of its innovation and economic competitiveness strategy for the next decade. The S$1 billion commitment (roughly US$779 million) sits within a broader ecosystem reshaped by last year’s S$37 billion Research, Innovation, 

In [230]:
def anthropic_research():
    research_ouput_anthropic = []
    for i in system_prompt:
        response = client_anthropic.messages.create(
            model = "claude-haiku-4-5-20251001",
            max_tokens = 4096,
            messages = [
                {
                    "role": "user",
                    "content": f"{i}"
                }
            ],
            tools = [
                {
                    "type": "web_search_20250305",
                    "name": "web_search",
                    "max_uses": 5
                }
            ]
        )

        research_ouput_anthropic.append(response.content)

    return research_ouput_anthropic

In [231]:
def extract_final_text(blocks):
    last_tool_idx = -1
    for i,b in enumerate(blocks):
        if getattr(b, "type", None) in ("server_tool_use", "web_search_tool_result"):
            last_tool_idx = i

    text_parts = []
    for b in blocks[last_tool_idx + 1:]:
        if getattr(b, "type", None) == "text":
            text_parts.append(b.text)

    return "\n".join(text_parts).strip()

In [233]:
def extract_sources(blocks):
    sources = []

    for b in blocks:
        if getattr(b, "type", None) == "web_search_tool_result":
            for r in getattr(b, "results", []):
                sources.append({
                    "title": r.get("title"),
                    "url": r.get("url"),
                    "source": r.get("source")
                })

    return sources

In [244]:
def final_claude_responses():
    results = anthropic_research()
    
    claude_responses = []
    
    for blocks in results:
        story = extract_final_text(blocks)          
        sources = extract_sources(blocks)           

        
        sources_text = "\n".join(
            f"- {s.get('title', '(no title)')}\n  {s.get('url', '')}"
            for s in sources
            if s.get("url")
        ).strip()

        
        payload = story
        if sources_text:
            payload += "\n\nSOURCES:\n" + sources_text

    
        if payload.strip():
            claude_responses.append(payload)

    return claude_responses

In [245]:
final_claude_responses()

['Based on my comprehensive research, here is a well-structured analysis of Singapore\'s AI investment announcement:\n\n---\n\n## **SINGAPORE\'S S$1 BILLION AI RESEARCH INVESTMENT (2026-2030): COMPREHENSIVE ANALYSIS**\n\n### **I. THE ANNOUNCEMENT**\n\n#### **Core Details**\n\nSingapore\'s government announced that it will invest more than S$1 billion (about $778.8 million) in public artificial intelligence research through 2030\n, announced on January 24, 2026 by the Ministry of Digital Development and Information.\n\n#### **Investment Allocation**\n\nThe government will invest in specific priority areas of research, such as building responsible and resource-efficient AI, and in developing the nation\'s talent from pre-university to faculty\n. \nSome funds will also go towards building capabilities to support the adoption and application of AI by industries\n.\n\n---\n\n### **II. RESEARCH PRIORITIES**\n\n#### **Key Focus Areas**\n\nThe funding will be directed toward key research prior

In [282]:
def claude_article_builder(researched_data):
    final_news_article = []
    for i in researched_data:
        response = client_anthropic.messages.create(
            model = "claude-sonnet-4-5",
            max_tokens = 1000,
            messages = [{
                "role": "user",
                "content":  f"""
                You are a professional tech journalist for "The AI Times" newspaper.
                
                Your task: Write a comprehensive news article covering today's AI developments.
                Entire news story and its details: {i}
                
                INSTRUCTIONS:
                1. Write a newspaper-quality article (800-1000 words)
                2. Create an engaging headline
                3. Use journalistic style: objective, clear, professional and opinionated
                4. Include all the sources & links at the end and not ANY sources or links during the article (THIS IS NON NEGOTIABLE)
                
                You are ONLY supposed to refer to the sources and links and NOT supposed to hallucinate. 
                
                However, the opinion you have be your own but it should be based on the above article. 
            """
            }]
        )


        final_output = response.content[0].text

        json_format = json_formatter(final_output, "claude")

        final_news_article.append(json_format)

    return final_news_article

In [269]:
a = ['Based on my comprehensive research, here is a well-structured analysis of Singapore\'s AI investment announcement:\n\n---\n\n## **SINGAPORE\'S S$1 BILLION AI RESEARCH INVESTMENT (2026-2030): COMPREHENSIVE ANALYSIS**\n\n### **I. THE ANNOUNCEMENT**\n\n#### **Core Details**\n\nSingapore\'s government announced that it will invest more than S$1 billion (about $778.8 million) in public artificial intelligence research through 2030\n, announced on January 24, 2026 by the Ministry of Digital Development and Information.\n\n#### **Investment Allocation**\n\nThe government will invest in specific priority areas of research, such as building responsible and resource-efficient AI, and in developing the nation\'s talent from pre-university to faculty\n. \nSome funds will also go towards building capabilities to support the adoption and application of AI by industries\n.\n\n---\n\n### **II. RESEARCH PRIORITIES**\n\n#### **Key Focus Areas**\n\nThe funding will be directed toward key research priorities, including building responsible, trustworthy and resource-efficient AI systems\n. \nSpecifically, Singapore needs to prioritise fundamental research in Responsible AI, Resource-efficient AI and Reasoning AI\n.\n\n#### **AI Safety and Governance**\n\nThe Singapore Consensus on Global AI Safety Research Priorities organises AI safety research domains into three broad buckets: creating trustworthy AI systems (Development), evaluating AI systems\' risks (Assessment), and monitoring and intervening after deployment (Control)\n.\n\n---\n\n### **III. TALENT DEVELOPMENT INITIATIVES**\n\n#### **Multi-Level Education Pipeline**\n\nSingapore aims to triple its AI practitioner pool from 4,500 to 15,000 by 2029\n. \nGovernment initiatives include the S$20 million SG Digital Scholarship funding AI-related education and overseas internships over three years. The TechSkills Accelerator has placed and trained 17,000 locals in AI and tech roles since 2016, while upskilling 231,000 individuals. New programs include the S$7 million AI Accelerated Master\'s Programme and partnerships creating 100 AI Centres of Excellence with companies\n.\n\n#### **AI Apprenticeship Programme (AIAP)**\n\nAIAP is a 9-month, full-time, in-person intensive programme with programme fees fully subsidized by the Singapore government, including a monthly stipend of $3,500 – $5,500 per month\n. \nThe program has nearly a 100% placement rate of students into the role they were trained for\n.\n\n#### **School-Level AI Literacy**\n\nAI Singapore has launched an "AI Student Outreach Programme" which aims to promote AI literacy and proficiency among students from local Secondary Schools, Integrated Programme (IP) Schools, Junior Colleges, Institutes of Technical Education, Polytechnics and Universities\n.\n\n---\n\n### **IV. HISTORICAL CONTEXT & PRIOR INVESTMENTS**\n\n#### **Previous Government Commitments**\n\nIn 2024, the country set aside S$500 million to secure high-performance computing infrastructure needed for AI innovation across both public and private sectors\n. \nThe government has also committed more than S$500 million to AI research and development through AI Singapore, a national programme designed to deepen AI capabilities\n.\n\n#### **HPC Infrastructure Funding**\n\nSingapore will commit S$270 million to develop its national supercomputing infrastructure and strengthen the National Supercomputing Centre (NSCC) Singapore\'s capabilities to support local research\n.\n\n#### **Previous Strategic Investments**\n\nIn November 2019, the government was investing SGD500 million (around USD364 million) to fund AI projects under the Research, Innovation and Enterprise 2020 plan\n.\n\n---\n\n### **V. SECTOR-SPECIFIC AI APPLICATIONS**\n\n#### **Healthcare**\n\nThe Ministry of Health\'s S$200 million Health Innovation Fund over five years supports AI-driven transformation in healthcare. SingHealth\'s Note Buddy supports over 2,100 healthcare workers, creating 16,000+ clinical notes with real-time transcription in English, Mandarin, Malay, and Tamil\n.\n\n#### **Financial Services**\n\nDBS Bank operates over 800 AI models across 350+ use cases, delivering S$750 million in economic value in 2024 and expecting to exceed S$1 billion by 2025\n.\n\n---\n\n### **VI. REGIONAL & INTERNATIONAL LEADERSHIP**\n\n#### **ASEAN Coordination**\n\nSingapore has invested significant efforts in bringing together Southeast Asia counterparts to make progress on building a trusted AI ecosystem in ASEAN. This is evident in Singapore\'s leadership in chairing ASEAN\'s AI Governance Working Group, which has been officially designated to oversee and coordinate AI projects in ASEAN\n.\n\n#### **Global AI Governance**\n\nSingapore hosted the SCAI: International Scientific Exchange on AI Safety in April 2025, gathering more than 100 minds from academia, industry, and government. The follow-up report, "The Singapore Consensus on Global AI Safety Research Priorities," was published in May 2025\n.\n\n#### **Regional Language Model Development**\n\nIn 2023, researchers released an open-source large language model called Southeast Asian Languages in One Network (Sea-Lion), backed by S$70 million in funding. A more recent version of Sea-Lion was released in October 2025, built on Alibaba\'s Qwen foundation model with improvements in Burmese, Filipino, Indonesian, Malay, Tamil, Thai and Vietnamese\n.\n\n---\n\n### **VII. STRATEGIC POSITIONING & COMPETITIVENESS**\n\n#### **Global Ranking**\n\nTortoise Media\'s Global AI Index continues to rank Singapore in third place, below the U.S. and China, in respect of level of investment, innovation and implementation of AI. Singapore is described as "Asia\'s most dynamic AI hub after China" and having "made big advancements on absolute metrics, especially on AI research and development"\n.\n\n#### **Government R&D Intensity**\n\nGovernment supported R&D spending as a percentage of GDP now exceeds the United States eighteen-fold\n.\n\n#### **Startup Ecosystem**\n\nSingapore\'s ecosystem includes 650 AI startups, with 230 having secured funding—a concentration that captures 91.1% of Southeast Asia\'s deep tech funding\n.\n\n---\n\n### **VIII. GOVERNANCE & TRUST FRAMEWORKS**\n\n#### **AI Governance Initiatives**\n\nThe AI Verify Testing Framework helps companies assess the responsible implementation of their AI system against 11 internationally recognised AI governance principles. While the Traditional AI version has been available since 2022, it has now been updated to include considerations for Generative AI applications\n.\n\n#### **Enterprise Readiness**\n\nAt Budget 2025, the Ministry of Finance announced that it has set aside up to $150 million for the Enterprise Compute Initiative (ECI)\n, designed to support Singapore-based companies in AI adoption.\n\n---\n\n### **IX. KEY SOURCES**\n\n- **Reuters (Jan 24, 2026)** - Original announcement\n- **Smart Nation Singapore** - National AI Strategy details\n- **AI Singapore** - Talent development and research programs\n- **Center for Security and Emerging Technology (CSET)** - Singapore AI progress analysis\n- **National Supercomputing Centre Singapore** - HPC infrastructure updates\n- **IMDA (Infocomm Media Development Authority)** - Talent scaling initiatives\n- **Introl Blog** - Comprehensive AI ecosystem overview\n- **IAPP** - AI governance frameworks\n\n---\n\nThis investment represents a continuation of Singapore\'s multi-year strategy to establish itself as a global AI hub while emphasizing responsible AI development, regional collaboration, and comprehensive talent cultivation across education levels.']

claude_article_builder(a)

[{'model': 'openai',
  'headline': 'Singapore Doubles Down on AI Dominance with $778M Research Investment Through 2030',
  'Content': "SINGAPORE — In its most ambitious artificial intelligence push yet, Singapore's government announced a sprawling S$1 billion ($778.8 million) investment in public AI research extending through 2030, signaling the city-state's determination to cement its position as Asia's premier AI hub outside China. The January 24, 2026 announcement by the Ministry of Digital Development and Information represents not merely an incremental funding increase, but a strategic doubling-down on AI leadership at a moment when global competition for technological supremacy has never been fiercer.\n\nThe investment arrives as Singapore continues leveraging its already formidable third-place global ranking in AI capabilities—trailing only the United States and China—into tangible economic and strategic advantages. With 650 AI startups capturing 91.1% of Southeast Asia's deep t

In [277]:
def json_formatter(final_output, model):

    json_format = []
    
    response_json = client.chat.completions.create(
            model = "gpt-4o-mini",
            messages = [
                {"role": "system", "content": f"""Your job is to look at the entire output of 
                {final_output} and convert into a STRICT JSON format
                
                Headline is pretty straightforward to determine
                Content: needs to be the full content after the headline
                Sources: are the links and sources from which the article is created

                YOU ARE NOT SUPPOSED TO UPDATE ANYTHING ON YOUR OWN, YOU NEED TO JUST TRANSFORM THE CONTENT INTO A JSON FORMAT
                """},
                {"role": "assistant", "content": f""" 
                {{
                  "model": "{model}",
                  "headline": "<headline here>",
                  "content": "<full article content here>",
                  "sources": ["<source here>"]
                }}
                """}
            ]
        )

    raw_response = response_json.choices[0].message.content
    
    clean_json = raw_response.replace("```json", "").replace("```", "").strip()

    article_data_cleaned = json.loads(clean_json)

    json_format.append(article_data_cleaned)

    return json_format